In [ ]:
### WARNING:

# Memory Issue on Free Colab Accounts
#
# Free Colab accounts provide approximately 12GB of RAM (CPU). When loading
# large graphs, you might encounter this error:
#
# "Your session crashed after using all available RAM."
#
# This indicates the session ended due to exceeding all the available RAM memory.
# To resolve this:
#
# 1. Run the setup cell (the first cell in the notebook), followed by the cell
#    that caused the error.
# 2. If the problem persists, go to Runtime > Restart runtime, then rerun the
#    setup cell, followed by the cell that caused the error.
#
# Tip: For additional memory, consider upgrading to a Colab Pro account.

### SETUP CELL - RUN THIS FIRST
!pip install rdflib
import rdflib
from rdflib import Graph

In [2]:
#### REQ1 #####
### The MMKG must conceptualize the food domain at a fine-grained level.

### YOU HAVE TO LOAD THE CORRESPONDING GRAPH, OTHERWISE RELATED QUERIES WILL NOT WORK

path_to_recipes1m_usda = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-usda-foods-nutrients-recipe1m.ttl"
recipes1m_usda_graph = Graph()
recipes1m_usda_graph.parse(path_to_recipes1m_usda, format="ttl")

# It seems that RDFLib is not able to import the ABox of fuskg-usda-foods-nutrients.ttl (and indirectly the tbox of usda) when doing parsing, then we have to explicitly load it
path_to_usda_tbox = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/TBox/fuskg-usda.ttl"
usda_tbox_graph = Graph()
usda_tbox_graph.parse(path_to_usda_tbox, format="ttl")

path_to_usda_fn = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-usda-foods-nutrients.ttl"
usda_fn_graph = Graph()
usda_fn_graph.parse(path_to_usda_fn, format="ttl")

graph_req1 = recipes1m_usda_graph + usda_tbox_graph + usda_fn_graph

print("DONE")

DONE


In [30]:
## CQ1-1 ##
# Which are the ingredients of recipe X?

recipe_X = "Yogurt Parfaits"
q1_1 = graph_req1.query(
    """
    SELECT ?foodLabel
    WHERE {{
        ?recipe a fsreci:Recipe1m ;
                rdfs:label "{}" ;
                fsreci:hasRecipeFood ?recipefood .
        ?recipefood fsreci:hasFood ?food .
        ?food rdfs:label ?foodLabel .
    }}
""".format(recipe_X)
)

print("Recipe {} has the following Ingredients:\n".format(recipe_X))
for result in q1_1:
    print("- Ingredient: {}".format(result[0]))

Recipe Yogurt Parfaits has the following Ingredients:

- Ingredient: Strawberries, raw
- Ingredient: Yogurt, Greek, plain, nonfat (Includes foods for USDA's Food Distribution Program)
- Ingredient: Cereals ready-to-eat, granola, homemade


In [31]:
## CQ1-2 ##
# Which are the nutrients of the food Y?

food_Y = "Strawberries, raw"
q1_2 = graph_req1.query(
    """
    SELECT ?nutrClassLabel
    WHERE {{
        ?food rdfs:label "{}"@en ;
              fscore:hasNutrient ?nutrInst .
        ?nutrInst a ?nutrClass .
        ?nutrClass rdfs:label ?nutrClassLabel .
        FILTER(STRSTARTS(str(?nutrClass), "https://w3id.org/fuskg/usda#usda-nut")) .
    }}
""".format(food_Y)
)

print("Food {} has the following Nutrients:\n".format(recipe_X))
for result in q1_2:
    print("- Nutrient: {}".format(result[0]))

Food Yogurt Parfaits has the following Nutrients:

- Nutrient: Tocopherol, gamma
- Nutrient: Fatty acids, total polyunsaturated
- Nutrient: Vitamin E (alpha-tocopherol)
- Nutrient: Tryptophan
- Nutrient: Magnesium, Mg
- Nutrient: Lactose
- Nutrient: Arginine
- Nutrient: Glutamic acid
- Nutrient: Vitamin K (phylloquinone)
- Nutrient: PUFA 18:4
- Nutrient: SFA 12:0
- Nutrient: Potassium, K
- Nutrient: Copper, Cu
- Nutrient: Sodium, Na
- Nutrient: Tocotrienol, alpha
- Nutrient: Methionine
- Nutrient: Lycopene
- Nutrient: Iron, Fe
- Nutrient: Leucine
- Nutrient: Vitamin D (D2 + D3)
- Nutrient: Tocopherol, beta
- Nutrient: Folate, total
- Nutrient: Vitamin K (Dihydrophylloquinone)
- Nutrient: Tyrosine
- Nutrient: Sugars, Total
- Nutrient: PUFA 18:3
- Nutrient: Maltose
- Nutrient: Vitamin D (D2 + D3), International Units
- Nutrient: Sucrose
- Nutrient: Total lipid (fat)
- Nutrient: Vitamin C, total ascorbic acid
- Nutrient: PUFA 22:6 n-3 (DHA)
- Nutrient: Phytosterols
- Nutrient: Cryptoxanth

In [32]:
## CQ1-3 ##
# How many calories does food Z have?

food_Z = "Mushrooms, portabella, raw"
q1_3 = graph_req1.query(
    """
    SELECT ?food ?amountCalories
    WHERE {{
        ?food rdfs:label "{}"@en ;
              fsfood:amountCalories ?amountCalories .
    }}
""".format(food_Z)
)

for result in q1_3:
    print("Food {} - Amount of calories: {}".format(food_Z, result[1]))

Food Mushrooms, portabella, raw - Amount of calories: 22.0


In [33]:
## CQ1-4 ##
# Which are the ingredients of recipe X that contain nutrient Y?

recipe_X = "Yogurt Parfaits"
nutrient_Y = "Sugars, Total"
q1_4 =  graph_req1.query(
    """
    SELECT ?foodLabel
    WHERE {{
        ?recipe a fsreci:Recipe1m ;
                 rdfs:label "{}" ;
                 fsreci:hasRecipeFood ?recipeFood .
        ?recipeFood fsreci:hasFood ?food .
        ?food rdfs:label ?foodLabel ;
            fscore:hasNutrient ?nutrInst .
        ?nutrInst a ?nutrClass .
        ?nutrClass rdfs:label "{}"@en .
    }}
    """.format(recipe_X, nutrient_Y)
)

for result in q1_4:
    print("Food {} of recipe {} contains nutrient {}".format(result[0], recipe_X, nutrient_Y))

Food Strawberries, raw of recipe Yogurt Parfaits contains nutrient Sugars, Total
Food Yogurt, Greek, plain, nonfat (Includes foods for USDA's Food Distribution Program) of recipe Yogurt Parfaits contains nutrient Sugars, Total
Food Cereals ready-to-eat, granola, homemade of recipe Yogurt Parfaits contains nutrient Sugars, Total


In [9]:
#### REQ2 ####
### The MMKG must provide a comprehensive list of activities that a person can perform. For each activity,
### it must be provided the knowledge required to understand the effort necessary to complete the activity
### to enable the inference of how much of each food defined under REQ1 is necessary to fulfill the activity.

### YOU HAVE TO LOAD THE CORRESPONDING GRAPH, OTHERWISE RELATED QUERIES WILL NOT WORK

path_to_activity_graph = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-activities.ttl"
activity_graph = Graph()
activity_graph.parse(path_to_activity_graph, format="ttl")

<Graph identifier=Ne551ca6e265347ce9bfa44a989df06a4 (<class 'rdflib.graph.Graph'>)>

In [34]:
## CQ2-1 ##
# Which is the effort needed to fulfill the activity X?

activity_X = "Salsa dancing"
q2_1 = activity_graph.query(
    """
    SELECT ?metValue
    WHERE {{
        ?activity a fsacti:Dancing ;
                 rdfs:label "{}"@en ;
                 fsacti:hasMETValue ?metValue .
    }}
    """.format(activity_X)
)

for result in q2_1:
    print("Activity {} has MET Value {}".format(activity_X, result[0]))

Activity Salsa dancing has MET Value 4.5


In [35]:
## CQ2-2 ##
# How many calories are consumed when performing activity Y for N minutes?

activity_Y = "mountain climbing"
N = 30
q2_2 = activity_graph.query(
    """
    SELECT ?KcalValue (?KcalValue * {} AS ?KcalValue30m)
    WHERE {{
        ?activity a fsacti:Sports ;
                 rdfs:label "{}"@en ;
                 fsacti:hasUnitKCalValue ?KcalValue .
    }}
    """.format(N, activity_Y)
)

for result in q2_2:
    print("Activity {} has {} Kcal per minute and when performed for {} minutes {} Kcal are consumed".format(activity_Y, round(float(result[0]), 2), N, round(float(result[1]), 2)))

Activity mountain climbing has 0.13 Kcal per minute and when performed for 30 minutes 4.0 Kcal are consumed


In [36]:
## CQ2-3 ##
# Which is the most intense activity of category C?

category_C = "WaterActivities"
q2_3 = activity_graph.query(
    """
    SELECT ?activityLabel ?maxMetValue
    WHERE {{
        {{
            SELECT (MAX(?metValue) as ?maxMetValue)
            WHERE {{
                ?activity a fsacti:{} ;
                         fsacti:hasMETValue ?metValue .
            }}
        }}
        ?activity a fsacti:{} ;
                 rdfs:label ?activityLabel ;
                 fsacti:hasMETValue ?maxMetValue .
        FILTER(LANG(?activityLabel) = 'en')
    }}
    """.format(category_C, category_C)
)
for result in q2_3:
    print("The most intense activity of category {} is {} with MET value {}".format(category_C, result[0], result[1]))

The most intense activity of category WaterActivities is free diving, vigorous effort with MET value 15.8
The most intense activity of category WaterActivities is scuba diving, vigorous effort with MET value 15.8


In [37]:
## CQ2-4 ##
# Which are the activities of category C that consume at least 0.1 kcal?

q2_4 = activity_graph.query(
    """
    SELECT ?activityLabel ?KcalValue
    WHERE {{
        ?activity a fsacti:{} ;
                 rdfs:label ?activityLabel ;
                 fsacti:hasUnitKCalValue ?KcalValue .
        FILTER(LANG(?activityLabel) = 'en' && ?KcalValue >= 0.1)
    }}
    ORDER BY ASC(?activityLabel)
    """.format(category_C)
)
for result in q2_4:
    print("Activity {} consumes {}".format(result[0], round(float(result[1]),2)))

Activity backstroke swimming competition consumes 0.16
Activity backstroke swimming training consumes 0.16
Activity backstroke swimming, moderate effort consumes 0.16
Activity breaststroke swimming competition consumes 0.16
Activity breaststroke swimming training consumes 0.16
Activity breaststroke swimming, moderate effort consumes 0.17
Activity butterfly swimming consumes 0.23
Activity canoeing crew in competition consumes 0.2
Activity crawl swimming, vigorous effort consumes 0.17
Activity free diving, light effort consumes 0.12
Activity free diving, moderate effort consumes 0.2
Activity free diving, vigorous effort consumes 0.26
Activity freestyle swimming, moderato effort consumes 0.14
Activity freestyle swimming, vigorous effort consumes 0.16
Activity kayaking competition consumes 0.21
Activity kayaking, vigorous effort consumes 0.21
Activity kitesurfing, moderate effort consumes 0.18
Activity kitesurfing, vigorous effort consumes 0.23
Activity open water swimming (lake, sea, rive

In [14]:
#### REQ3 #####
### The MMKG must include the model of the barriers that may affect a person and how such barriers obstacle
### the fulfillment of specific activities, the consumption of specific foods, or, in general, the following of
### specific guidelines.

### YOU HAVE TO LOAD THE CORRESPONDING GRAPH, OTHERWISE RELATED QUERIES WILL NOT WORK
path_to_user_example = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/example/fuskg-user-example.ttl"
user_example_graph = Graph()
user_example_graph.parse(path_to_user_example, format="ttl")

# It seems that RDFLib is not able to import the ABox of fuskg-usda-foods-nutrients.ttl, fuskg-activities.ttl (and indirectly the tbox of barriers) when doing parsing, then we have to explicitly load them
path_to_usda_fn = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-usda-foods-nutrients.ttl"
usda_fn_graph = Graph()
usda_fn_graph.parse(path_to_usda_fn, format="ttl")

path_to_activity= "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-activities.ttl"
activity_graph = Graph()
activity_graph.parse(path_to_activity, format="ttl")

path_to_barriers = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/TBox/fuskg-barriers.ttl"
barriers_graph = Graph()
barriers_graph.parse(path_to_barriers, format="ttl")

req3_graph = user_example_graph + usda_fn_graph + activity_graph + barriers_graph

print("Done")

Done


In [38]:
## CQ3-1 ##
# Why user U cannot perform activity X?

user_U = "PatientX"
activity_Z = "playing rugby, vigorous effort"

q3_1 = req3_graph.query(
    """
    SELECT ?barrier ?barrierType
    WHERE {{
      ?user rdfs:label "{}"@en ;
            fsbarr:hasDiagnosis ?diagnosis .
      ?diagnosis fsbarr:hasDisease ?disease.
      ?barrier a ?barrierType ;
               fsbarr:isTriggeredBy ?diagnosis;
               fscore:affectsPerformingOf ?activity.
      ?activity rdfs:label "{}"@en .
      ?barrierType rdfs:subClassOf fscore:Barrier .
    }}
    """.format(user_U, activity_Z))

for result in q3_1:
    barrier_name = result[0].split('#')[-1]
    barrier_type_name = result[1].split('#')[-1]

    print("Barrier {} of Barrier Type {} prevents user {} to {}".format(barrier_name, barrier_type_name, user_U, activity_Z))

Barrier ChestPain of Barrier Type HealthBarrier prevents user PatientX to playing rugby, vigorous effort


In [39]:
## CQ3-2 ##
# Why user U cannot consume food Y?

user_U = "PatientX"
foodY = "Butter, salted"

q3_2 = req3_graph.query(
    """
    SELECT ?disease ?diseaseRiskLevel
    WHERE {{
      ?user rdfs:label "{}"@en ;
            fsbarr:hasDiagnosis ?diagnosis .
      ?diagnosis fsbarr:hasDisease ?disease.
      ?diseaseRiskLevel a fsdise:DiseaseRiskLevel ;
            fsbarr:hasDisease ?disease ;
            fsuser:hasFood ?food .
      ?food rdfs:label "{}"@en .
    }}
    """.format(user_U, foodY)
)

for result in q3_2:
    # Extract local names (after the last '#')
    disease_name = result[0].split('#')[-1]
    disease_risk_level_name = result[1].split('#')[-1]

    # Print the extracted names
    print("User {} cannot consumed food {} because he has {} disease \n(i.e., there exists a DiseaseRiskLevel {} correlated to the disease)".
          format(user_U, foodY, disease_name, disease_risk_level_name))

User PatientX cannot consumed food Butter, salted because he has Cardiovascular disease 
(i.e., there exists a DiseaseRiskLevel DRL-USDAFOOD-00001-C correlated to the disease)


In [40]:
## CQ3-3 ##
# Which are the barriers preventing the fulfillment of activity Z?

activity_Z = "cycling in mountain-bike, vigorous effort"

q3_3 = req3_graph.query("""
    SELECT ?barrier ?barrierType
    WHERE {{
      ?barrier a ?barrierType ;
                fscore:affectsPerformingOf ?activity .
      ?activity rdfs:label "{}"@en ;
                a fsacti:Bicycling .
      ?barrierType rdfs:subClassOf* fscore:Barrier .
    }}
""".format(activity_Z))


for result in q3_3:
    # Extract local names (after the last '#')
    barrier_name = result[0].split('#')[-1]
    barrier_type_name = result[1].split('#')[-1]

    print("Barrier {} of type {} prevents {}".format(barrier_name, barrier_type_name, activity_Z))

Barrier BikeCost of type EquipmentCost prevents cycling in mountain-bike, vigorous effort


In [6]:
#### REQ4 #####
### The MMKG must support the modeling of multi-modal knowledge since a MMKG may be exploited to better support
### users’ education tasks and enable knowledge injection tasks into large foundational models.

### YOU HAVE TO LOAD THE CORRESPONDING GRAPHS, OTHERWISE RELATED QUERIES WILL NOT WORK

# Due to size of fuskg-multimodal-recipe1m total loading time is > 5 minutes
path_to_recipes1m_multimodal = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-multimodal-recipe1m.ttl"
path_to_tasty_multimodal = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-multimodal-tasty.ttl"
path_to_notcooking_example = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/example/fuskg-notcooking-example.ttl"

multimodal_graph_recipe1m = Graph()
multimodal_graph_tasty = Graph()
notcooking_example_graph = Graph()

multimodal_graph_recipe1m.parse(path_to_recipes1m_multimodal, format="ttl")
multimodal_graph_tasty.parse(path_to_tasty_multimodal, format="ttl")
notcooking_example_graph.parse(path_to_notcooking_example, format="ttl")

# It seems that RDFLib is not able to import the ABox of fuskg-activities.ttl, then we have to explicitly load it
path_to_activity= "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-activities.ttl"
activity_graph = Graph()
activity_graph.parse(path_to_activity, format="ttl")

combined_graph = notcooking_example_graph + activity_graph

print("Done")

Done


In [7]:
## CQ4-1 ##
# Where is it possible to find a picture of recipe X?
recipe_X = "BlueberryIceCream"
q4_1 = multimodal_graph_recipe1m.query(
    """
    SELECT ?recipe ?modalDescriptor ?resourceURL ?pathInArtifact
    WHERE {{
        ?recipe a fsreci:Recipe1m ;
                rdfs:label "{}" .
        ?recipe fsmumo:hasModalDescriptor ?modalDescriptor .
        ?modalDescriptor fsmumo:resourceURL ?resourceURL ;
                         fsmumo:hasPathInArtifact ?pathInArtifact .
    }}
""".format(recipe_X)
)

for result in q4_1:
    print("Recipe {} can be downloaded at ResourceURL: {}".format(recipe_X, result[2]))
    print("and the path inside the archive is: {}".format(result[3]))

Recipe BlueberryIceCream can be downloaded at ResourceURL: https://fbk.sharepoint.com/:f:/s/IDA/EvsBg8UfMy5Cqh6lbrBAVn4BCWfBJq7Jks_6sRDD_kArAQ?e=oamRuY
and the path inside the archive is: ./recipe1m_nutritional/Recipe1m-25034_BlueberryIceCream/70b927a328.jpg
Recipe BlueberryIceCream can be downloaded at ResourceURL: https://fbk.sharepoint.com/:f:/s/IDA/EvsBg8UfMy5Cqh6lbrBAVn4BCWfBJq7Jks_6sRDD_kArAQ?e=oamRuY
and the path inside the archive is: ./recipe1m_nutritional/Recipe1m-25034_BlueberryIceCream/33ca57d6af.jpg
Recipe BlueberryIceCream can be downloaded at ResourceURL: https://fbk.sharepoint.com/:f:/s/IDA/EvsBg8UfMy5Cqh6lbrBAVn4BCWfBJq7Jks_6sRDD_kArAQ?e=oamRuY
and the path inside the archive is: ./recipe1m_nutritional/Recipe1m-25034_BlueberryIceCream/9e227c84c0.jpg
Recipe BlueberryIceCream can be downloaded at ResourceURL: https://fbk.sharepoint.com/:f:/s/IDA/EvsBg8UfMy5Cqh6lbrBAVn4BCWfBJq7Jks_6sRDD_kArAQ?e=oamRuY
and the path inside the archive is: ./recipe1m_nutritional/Recipe1m-250

In [8]:
## CQ4-2 ##
# Where is it possible to download a video showing how to perform activity Y? --- ongoing finding data
activity_Y = "changing a light bulb"
q4_2 = combined_graph.query(
    """
    SELECT ?activity ?modalDescriptor ?resourceURL
    WHERE {{
        ?activity a fsacti:HomeActivity ;
                  rdfs:label "{}"@en .
        ?activity fsmumo:hasModalDescriptor ?modalDescriptor .
        ?modalDescriptor fsmumo:resourceURL ?resourceURL .
    }}
""".format(activity_Y)
)

for result in q4_2:
    print("Activity {} can be downloaded at ResourceURL: {}".format(activity_Y, result[2]))

Activity changing a light bulb can be downloaded at ResourceURL: https://www.youtube.com/watch?v=Y573xAR_eMw


In [9]:
## CQ4-3 ##
# Where is it possible to find a video showing how to prepare recipe Z?

recipe_Z = "chocolate-coconut-almond-balls"
q4_3 = multimodal_graph_tasty.query(
    """
    SELECT ?recipe ?modalDescriptor ?resourceURL ?pathInArtifact
    WHERE {{
        ?recipe a fsreci:Tasty ;
                rdfs:label "{}" .
        ?recipe fsmumo:hasModalDescriptor ?modalDescriptor .
        ?modalDescriptor fsmumo:resourceURL ?resourceURL ;
                         fsmumo:hasPathInArtifact ?pathInArtifact .
    }}
""".format(recipe_Z)
)

for result in q4_3:
    print("Recipe {} can be download at ResourceURL: {}".format(recipe_Z, result[2]))
    print("and the path inside the archive is: {}".format(result[3]))

Recipe chocolate-coconut-almond-balls can be download at ResourceURL: https://fbk.sharepoint.com/:f:/s/IDA/EoA_tdSP9MpCo_ebxoOY7bwBztKr9asvB_pqNIjJ48p32w?e=f1qg6M
and the path inside the archive is: /Tasty_Videos_Dataset/ALL_RECIPES_without_videos/chocolate-coconut-almond-balls/recipe_video.mp4


In [26]:
#### REQ5 #####
### Knowledge modeled under the requirements REQ1, REQ2, REQ3, and REQ4 must be associated with knowledge and
### data gathered from users’ input.

path_to_user_example = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/example/fuskg-user-example.ttl"
user_example_graph = Graph()
user_example_graph.parse(path_to_user_example, format="ttl")

# It seems that RDFLib is not able to import the ABox of fuskg-usda-foods-nutrients.ttl and fuskg-activities.ttl  when doing parsing, then we have to explicitly load them

path_to_usda_fn = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-usda-foods-nutrients.ttl"
usda_fn_graph = Graph()
usda_fn_graph.parse(path_to_usda_fn, format="ttl")

path_to_activity= "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-activities.ttl"
activity_graph = Graph()
activity_graph.parse(path_to_activity, format="ttl")

req5_graph = user_example_graph + usda_fn_graph + activity_graph
print("DONE")

DONE


In [41]:
## CQ5-1 ##
# What did the user U consume N days ago for breakfast?
# Suppose today is 24-05-31 and N=5

user_U = "PatientX"
q5_1 = req5_graph.query(
    """
    SELECT ?food ?foodLabel
    WHERE {{
        ?user rdfs:label "{}"@en ;
              fsuser:consumed ?consumedMeal .
        ?consumedMeal a fsuser:Breakfast ;
                      fsuser:hasTimestamp ?timestamp ;
                      fsuser:hasConsumedFood ?consumedFood .
        ?consumedFood fsuser:hasFood ?food .
        ?food rdfs:label ?foodLabel .
        FILTER (?timestamp > "2024-05-26T00:00:00"^^xsd:dateTime && ?timestamp < "2024-05-26T23:59:59"^^xsd:dateTime)
    }}
""".format(user_U)
)

print("User {} has consumed for breakfast at 26/05/24:\n".format(user_U))
for result in q5_1:
    print("- {}".format(result[1]))

User PatientX has consumed for breakfast at 26/05/24:

- Cereals ready-to-eat, FAMILIA
- Milk, fluid, nonfat, calcium fortified (fat free or skim)


In [42]:
## CQ5-2 ##
# How many times did user U perform activity Y last week?
# Suppose today is 24-05-28

user_U = "PatientX"
activity_Y = "archery sport"
q5_2 = req5_graph.query(
    """
    SELECT ?startActivity ?endActivity
    WHERE {{
        ?user rdfs:label "{}"@en ;
              fsuser:performed ?performedActivity.
        ?performedActivity fsuser:hasActivity ?activity ;
              fsuser:activityStartTimestamp ?startActivity ;
              fsuser:activityEndTimestamp ?endActivity .
        ?activity rdfs:label "{}"@en .
        FILTER (?startActivity > "2024-05-21T00:00:00"^^xsd:dateTime && ?endActivity < "2024-05-27T23:59:59"^^xsd:dateTime)
    }}
""".format(user_U, activity_Y)
)

print("User {} perfomed {} times the activity {} last week:\n".format(user_U, len(q5_2), activity_Y))
for result in q5_2:
    print("User {} performed activity {} from {} to {}".format(user_U, activity_Y, result[0], result[1]))

User PatientX perfomed 3 times the activity archery sport last week:

User PatientX performed activity archery sport from 2024-05-24T10:07:44 to 2024-05-24T12:37:27
User PatientX performed activity archery sport from 2024-05-22T15:07:14 to 2024-05-22T16:47:24
User PatientX performed activity archery sport from 2024-05-23T16:30:25 to 2024-05-23T17:40:04


In [43]:
## CQ5-3 ##
# Did user U consume food Y today?
# Suppose today is 24-05-31 and food_Y = Fish, tuna salad

user_U = "PatientX"
q5_3 = req5_graph.query(
    """
    SELECT ?foodLabel
    WHERE {{
        ?user rdfs:label "{}"@en ;
              fsuser:consumed ?consumedMeal .
        ?consumedMeal fsuser:hasTimestamp ?timestamp ;
                      fsuser:hasConsumedFood ?consumedFood .
        ?consumedFood fsuser:hasFood ?food .
        ?food rdfs:label ?foodLabel ;
        FILTER (
                ?foodLabel = "Fish, tuna salad"@en &&
                ?timestamp > "2024-05-31T00:00:00"^^xsd:dateTime && ?timestamp < "2024-05-31T23:59:59"^^xsd:dateTime)
    }}
""".format(user_U)
)


for result in q5_3:
    if str(result[0]) == "Fish, tuna salad":
        print("yes, user {} has consumed food: {}".format(user_U, result[0]))
    else:
        print("no, user {} has not consumed food Fish, tuna salad".format(user_U))


yes, user PatientX has consumed food: Fish, tuna salad


In [44]:
## CQ5-4 ##
# How many calories were consumed by user U yesterday?
# Suppose today is 24-05-31, so yesterdy is 24-05-30

user_U = "PatientX"
q5_4 = req5_graph.query(
    """
    SELECT ?foodLabel ?amountCalories (SUM(?amountCalories) AS ?totalCalories)
    WHERE {{
        ?user rdfs:label "{}"@en ;
              fsuser:consumed ?consumedMeal .
        ?consumedMeal fsuser:hasTimestamp ?timestamp ;
              fsuser:hasConsumedFood ?consumedFood .
        ?consumedFood fsuser:hasFood ?food .
        ?food fsfood:amountCalories ?amountCalories ;
        FILTER (?timestamp > "2024-05-30T00:00:00"^^xsd:dateTime && ?timestamp < "2024-05-30T23:59:59"^^xsd:dateTime)
    }}
""".format(user_U)
)

for result in q5_4:
    print("User {} has consumed {} calories yesterday".format(user_U, result.totalCalories))

User PatientX has consumed 666.0 calories yesterday


In [45]:
#### REQ6 #####
### A MMKG usable for creating a behavior change solution requires a set of guidelines driving the behavior change intervention.
path_to_user_example = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/example/fuskg-user-example.ttl"
user_example_graph = Graph()
user_example_graph.parse(path_to_user_example, format="ttl")

# It seems that RDFLib is not able to import the ABox of fuskg-usda-foods-nutrients.ttl and fuskg-activities.ttl  when doing parsing, then we have to explicitly load them

path_to_usda_fn = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-usda-foods-nutrients.ttl"
usda_fn_graph = Graph()
usda_fn_graph.parse(path_to_usda_fn, format="ttl")

path_to_activity= "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-activities.ttl"
activity_graph = Graph()
activity_graph.parse(path_to_activity, format="ttl")

req6_graph = user_example_graph + usda_fn_graph + activity_graph
print("DONE")

DONE


In [46]:
## CQ6-1 ##
# How many times is it possible to consume food X in a week?

user_U = "PatientX"
food_X = "Fish, tuna salad"

q6_1 = req6_graph.query(
    """
    SELECT ?monitoredValue ?operator
    WHERE {{
        ?monitoringRule a fsguid:MonitoringRule ;
                    fsguid:monitors ?user ;
                    fsguid:monitoredEntityType fscore:Food ;
                    fsguid:monitoredEntity ?monitoredEntity ;
                    fsguid:timing fstemp:Week ;
                    fsguid:hasMonitoredValue ?monitoredValue ;
                    fsguid:hasOperator ?operator .
        ?user rdfs:label "{}"@en .
        ?monitoredEntity rdfs:label "{}"@en .
    }}
""".format(user_U, food_X)
)

# greater includes also equal (so, it's >=)
for result in q6_1:
    print("Food {} can be consumed {} {} times in a week".format(food_X, result[1], result[0]))

Food Fish, tuna salad can be consumed Greater 1 times in a week


In [47]:
## CQ6-2 ##
# How many minutes of activity Y should be performed by user U in a single training session?

user_U = "PatientX"
activity_Y = "archery sport"

q6_2 = req6_graph.query(
    """
    SELECT ?monitoredValue ?operator
    WHERE {{
        ?user rdfs:label "{}"@en .
        ?monitoringRule a fsguid:MonitoringRule ;
                    fsguid:monitors ?user ;
                    fsguid:monitoredEntityType fsacti:Sports ;
                    fsguid:monitoredEntity ?monitoredEntity ;
                    fsguid:timing fstemp:Week ;
                    fsguid:hasMonitoredValue ?monitoredValue ;
                    fsguid:hasOperator ?operator .
        ?monitoredEntity rdfs:label "{}"@en .
    }}
""".format(user_U, activity_Y)
)

# greater includes also equal (so, it's >=)
for result in q6_2:
    print("User {} should performed {} {} min of activity {}".format(user_U, result[1], result[0], activity_Y))

User PatientX should performed Greater 150 min of activity archery sport


In [48]:
## CQ6-3 ##
# How much food Y can be eaten during a single meal?

user_U = "PatientX"
food_Y = "Tortellini, pasta with cheese filling, fresh-refrigerated, as purchased"

q6_3 = req6_graph.query(
    """
    SELECT ?monitoredValue ?operator
    WHERE {{
        ?monitoringRule a fsguid:MonitoringRule ;
                    fsguid:monitors ?user ;
                    fsguid:monitoredEntityType fscore:Food ;
                    fsguid:monitoredEntity ?monitoredEntity ;
                    fsguid:timing fstemp:Meal ;
                    fsguid:hasMonitoredValue ?monitoredValue ;
                    fsguid:hasOperator ?operator .
        ?user rdfs:label "{}"@en .
        ?monitoredEntity rdfs:label "{}"@en .
    }}
""".format(user_U, food_Y)
)

# greater includes also equal (so, it's >=)
for result in q6_3:
    print("User {} can consume {} {}g of {} in a Meal".format(user_U, result[1], result[0],  food_Y))

User PatientX can consume Equal 100g of Tortellini, pasta with cheese filling, fresh-refrigerated, as purchased in a Meal


In [51]:
## CQ6-4 ##
# Is it allowed to consume M grams of food Z in a single day?

user_U = "PatientX"
food_Y = "Yogurt, vanilla, low fat."
M = 50

q6_4 = req6_graph.query(
    """
    SELECT ?monitoredValue ?operator
    WHERE {{
        ?monitoringRule a fsguid:MonitoringRule ;
                    fsguid:monitors ?user ;
                    fsguid:monitoredEntityType fscore:Food ;
                    fsguid:monitoredEntity ?monitoredEntity ;
                    fsguid:timing fstemp:Day ;
                    fsguid:hasMonitoredValue ?monitoredValue ;
                    fsguid:hasOperator ?operator .
        ?user rdfs:label "{}"@en .
        ?monitoredEntity rdfs:label "{}"@en .
    }}
""".format(user_U, food_Y)
)

for result in q6_4:
    quantity_to_consume = result[0]
    print("User {} can consume {} {} grams of {} in a day".format(user_U, result[1], result[0],  food_Y))
    if quantity_to_consume >= M:
        print("\n so, user {} cannot consume {} grams of food {}".format(user_U, M, food_Y))
    else:
        print("\n so, user {} can consume {} grams of food {}".format(user_U, M, food_Y))

User PatientX can consume Equal 30 grams of Yogurt, vanilla, low fat. in a day

 so, user PatientX cannot consume 50 grams of food Yogurt, vanilla, low fat.


In [2]:
#### REQ7 #####
### The MMKG must include temporal knowledge to support the requirements above and enable temporal reasoning over the users’ collected knowledge.

### YOU HAVE TO LOAD THE CORRESPONDING GRAPH, OTHERWISE RELATED QUERIES WILL NOT WORK

path_to_tasty_rsteps = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-recipes-recipessteps-tasty.ttl"
tasty_rsteps_graph = Graph()
tasty_rsteps_graph.parse(path_to_tasty_rsteps, format="ttl")

path_to_temporal_tbox = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/TBox/fuskg-temporal.ttl"
temporal_tbox_graph = Graph()
temporal_tbox_graph.parse(path_to_temporal_tbox, format="ttl")

path_to_notcooking_example = "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/example/fuskg-notcooking-example.ttl"
notcooking_example_graph = Graph()
notcooking_example_graph.parse(path_to_notcooking_example, format="ttl")

# It seems that RDFLib is not able to import the ABox of fuskg-activities.ttl when doing parsing, then we have to explicitly load it
path_to_activity= "https://media.githubusercontent.com/media/IDA-FBK/FuS-KG/refs/heads/update-modules/ontology/ABox/fuskg-activities.ttl"
activity_graph = Graph()
activity_graph.parse(path_to_activity, format="ttl")

combined_graph = notcooking_example_graph + activity_graph

print("Done")

Done


In [3]:
## CQ7-1 ##
# Which is the step following step X during the preparation of recipe Y?

recipe_Y = "sofritas-burrito-bowl"
step_X = "add the soy sauce, chili powder, garlic powder, cumin, paprika, and cayenne."

q7_1 = tasty_rsteps_graph.query(
    """
    SELECT ?recipeStepAfterLabel
    WHERE {{
        ?recipe a fsreci:Tasty ;
                 rdfs:label "{}" ;
                 fstemp:hasFirstRecipeStep ?firstStep .
        ?firstStep fstemp:afterStep* ?recipeStepBefore .
        ?recipeStepBefore fstemp:hasStepDescription ?stepBeforeDescriptionStep .
        ?stepBeforeDescriptionStep fstemp:stepDescription "{}"^^xsd:string.
        ?recipeStepBefore fstemp:afterStep ?recipeStepAfter.
        ?recipeStepAfter fstemp:hasStepDescription ?StepAfterDescriptionStep .
        ?StepAfterDescriptionStep fstemp:stepDescription ?recipeStepAfterLabel .
    }}
    """.format(recipe_Y, step_X)
)

for result in q7_1:
    print("Step following '{}' is:\n\n {}".format(step_X, result[0]))

Step following 'add the soy sauce, chili powder, garlic powder, cumin, paprika, and cayenne.' is:

 continue to cook and break the tofu apart until browned and crumbly.


In [4]:
## CQ7-2 ##
# Which is the second movement to do when performing activity Z?

activity_Z = "changing a light bulb"

q7_2 = combined_graph.query(
    """
    SELECT ?secondMovementDescription
    WHERE {{
        ?activity a fsacti:HomeActivity ;
                 rdfs:label "{}"@en ;
                 fstemp:hasFirstRecipeStep ?firstStep .
        ?firstStep fstemp:afterStep ?secondStep .
        ?secondStep fstemp:hasStepDescription ?secondStepDescription .
        ?secondStepDescription fstemp:stepDescription ?secondMovementDescription.
    }}
    """.format(activity_Z)
)

for result in q7_2:
    print("Second movement of activity {} is: {}".format(activity_Z, result[0]))

Second movement of activity changing a light bulb is: Take out the old bulb


In [5]:
## CQ7-3 ##
# Which are the timespans used to build guidelines?

q7_3 = temporal_tbox_graph.query(
    """
    SELECT ?timespanLabel
    WHERE {{
      ?timespan rdfs:subClassOf fstemp:Timespan ;
                rdfs:label ?timespanLabel .
      FILTER(LANG(?timespanLabel) = "en")
    }}
    """
)

print("The timespans used to build guidelines are:")
for result in q7_3:
    print("- {}".format(result[0]))

The timespans used to build guidelines are:
- Day
- Week
